In [ ]:
#!pip install -U swifter

In [ ]:
# !pip install -U pandas
# !pip install -U seaborn

In [ ]:
#!sed -i ':a;N;$!ba;s/\,\n/\,/g' results/Cameron*

In [ ]:
import re
import pandas as pd
import analysis_relabel_funcs
from analysis import read_dfs
import torch
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from torch.nn import Softmax
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import pickle
import swifter
import time
%matplotlib inline

In [ ]:
MODEL_RELABEL = {
    'BERT-Jigsaw': ["not toxic", "toxic"], 
    'BERT-SBIC-offensive': ["Not", "Maybe", "Offensive"],
    'BERT-SBIC-targetcategory': ['none', 'body', 'culture', 'disabled', 'gender', 'race', 'social', 'victim'], # not used
    'BERT-eec-emotion': ['none', 'anger','fear', 'joy', 'sadness'], # not used
    'BERT-jigsaw-identityhate': ['Not', "Yes"],
    'BERT-jigsaw-severetoxic': ["Not", "Yes"],
    'BERT-mdgender-convai-binary': ["female", "male"],
    'BERT-mdgender-convai-ternary': ["female", "male", "neutral"],
    'BERT-mdgender-wizard': ["neutral", "female", "male"],
    'BERT-rtgender-opgender-annotations': ["man", "woman"] 
}

In [ ]:
def _softmax_and_relabel(predictions, categorical_labels):
    # takes in a torch.Tensor of predictions and a list of categorical labels, returns a tuple of (softmax tensor, categorical label)
    m = Softmax(dim=0)
    sm = m(predictions)
    return sm, categorical_labels[sm.argmax().item()]

def score(labels, predictions):
    softmax_preds, category = _softmax_and_relabel(predictions, labels)
    # scores are always the difference between the first and last
    return softmax_preds[-1] - softmax_preds[0], category

In [ ]:
TOXIC_MODELS = {"Cameron/BERT-Jigsaw", 
                'Cameron/BERT-SBIC-offensive', 
                #'Cameron/BERT-jigsaw-identityhate', 'Cameron/BERT-jigsaw-severetoxic'
               }
GENDER_MODELS = {'Cameron/BERT-mdgender-convai-binary', 'Cameron/BERT-mdgender-convai-ternary', 'Cameron/BERT-mdgender-wizard', 'Cameron/BERT-rtgender-opgender-annotations'}

In [ ]:
import glob
FILES = glob.glob("results/Cameron*")

In [ ]:
print(FILES[:3])

In [ ]:
eval_files = {}
toxic_files = []
gender_files = []

def create_tup(filename, model_name):
    return (model_name.replace("Cameron/", ""), filename)

for filename in FILES:
    s = filename[len("results/"):].split(":")
    
    model_name = s[0][:-3]
    model_name = model_name.replace("Cameron-", "Cameron/")
    eval_dataset_name = s[-1][3:-15]
    model_type = None
    if eval_dataset_name not in eval_files:
        eval_files[eval_dataset_name] = {"toxic": [], "gender": []}
    if model_name in TOXIC_MODELS:
        model_type = "TOXIC"
        eval_files[eval_dataset_name]["toxic"].append(create_tup(filename, model_name))
        toxic_files.append(filename)
    elif model_name in GENDER_MODELS:
        model_type = "GENDER"
        eval_files[eval_dataset_name]["gender"].append(create_tup(filename, model_name))
        gender_files.append(filename)
    else:
        print(f"Model type is not classified, please classify {model_name} with file {filename}")
        continue
    print(f"Model name: {model_name} | eval dataset: {eval_dataset_name}" )

In [ ]:
print(f"Found {len(toxic_files)} toxic results and {len(gender_files)} gender files")

In [ ]:
eval_files

In [ ]:
CACHE = {}
def get_cache(filename):
    if filename in CACHE:
        return CACHE[filename]
    return None

def store_cache(filename, data):
    CACHE[filename] = data
    
def cross_tab_gender_toxic(eval_file_split, eval_dataset_name, return_one = False):
    global CACHE
    CACHE = {}
    print(eval_file_split)
    eval_crosstabs = []
    count = 0.0
    tot = len(eval_file_split["toxic"]) * len(eval_file_split["gender"])
    print(tot)
    for toxic_tup in eval_file_split["toxic"]:
        toxic_model_name = toxic_tup[0]
        toxic_filename = toxic_tup[1]
        for gender_tup in eval_file_split["gender"]:
            gender_model_name = gender_tup[0]
            gender_filename = gender_tup[1]
            
            # for using sbc and rt_gender for conv_ai
            _, _, combined = read_dfs(toxic_filename, 
                                      gender_filename,
                                     suffixes=("_" + toxic_model_name, "_" + gender_model_name))
            pred_toxic = "predictions_" + toxic_model_name
            pred_gender = "predictions_" + gender_model_name
            
            print("relabeling toxic")
            s = get_cache(toxic_filename)
            if s is None:
                print(f"did not find {toxic_filename} in cache")
                p = partial(score, MODEL_RELABEL[toxic_model_name])
                s = combined[pred_toxic].swifter.apply(p)
                scores_col = f"scores_{toxic_model_name}"
                category_col = f"category_{toxic_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(toxic_filename, s)
            else:
                print(f"found toxic {toxic_filename} in cache")
            combined = combined.join(s)
            
            print("relabeling gender")
            s = get_cache(gender_filename)
            if s is None:
                print(f"did not find {gender_filename} in cache")
                p = partial(score, MODEL_RELABEL[gender_model_name])
                s = combined[pred_gender].swifter.apply(p)
                scores_col = f"scores_{gender_model_name}"
                category_col = f"category_{gender_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(gender_filename, s)
            else:
                print(f"found gender {gender_filename} in cache")
            combined = combined.join(s)
            
            
            eval_crosstabs.append({
                "toxic_model_name": toxic_model_name,
                "gender_model_name": gender_model_name,
                "toxic_attr": (f"scores_{toxic_model_name}", f"category_{toxic_model_name}"),
                "gender_attr": (f"scores_{gender_model_name}", f"category_{gender_model_name}"),
                "df": combined
            })
            count += 1.0
            print(f"Crossing {toxic_model_name} x {gender_model_name} for {eval_dataset_name}, finished {count/tot*100}%")
            if return_one:
                return eval_crosstabs
    return eval_crosstabs

In [ ]:
eval_files.keys()

In [ ]:
notebook_files = ['yahoo_answers_topics']
print(eval_files['yahoo_answers_topics'])
for eval_dataset_name in notebook_files:
    eval_f = eval_files[eval_dataset_name]
#     if eval_dataset_name != "conv_ai_3":
#         continue
#     if eval_dataset_name == "air_dialogue" or eval_dataset_name == "empathetic_dialogues":
#         print(f'ignoring {eval_dataset_name} for now')
#         continue
    print(f"Analyzing {eval_dataset_name}")
    try:
        ct = cross_tab_gender_toxic(eval_f, eval_dataset_name, return_one=False)
        pickle.dump((eval_dataset_name, ct), open(f"crosstabs-pickle-{eval_dataset_name}.p", "wb"))
        #crosstabs.append((eval_dataset_name, ct))

    except Exception as e:
        print(e)
        raise 
    

In [ ]:
ct